In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
#os.environ['CUDA_VISIBLE_DEVICES']='0,1'
#os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
print(torch.cuda.current_device())

0


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
torch.cuda.current_device()

0

In [10]:
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/Lu_asr_lstm4atthead-ctc2.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs', 8)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 8)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    solver = Solver(config, paras, mode)
    solver.load_data()
    solver.set_model()
    solver.exec()


In [11]:
main()


Using train mode

[INFO] Exp. name : Lu_asr_lstm4atthead-ctc2_sd0                                                            
[INFO] Loading data... large dataset may took a while.                                                     
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Load text encoder : CharacterTextEncoder
Interface for creating all kinds of dataset
import LuDataset as Dataset
[LuDataset] path: /mnt/usb/jason3/cv-corpus-11.0-2022-09-21/zh-TW/wavs, split: ['Ludev']
LuDataset Ludev found wav data: 22
text len: 22
remove None, then wav data: 22, text len: 22
[LuDataset] path: /mnt/usb/jason3/cv-corpus-11.0-2022-09-21/zh-TW/wavs, split: ['Lutrain']
LuDataset Lutrain found wav data: 885
text len: 885
remove None, then wav data: 881, text len: 881
[INFO] Data spec. | Corpus = Lu (from /mnt/usb/jason3/cv-corpus-11.0-2022-09-21/zh-TW/wavs)                
[INFO]            | Train sets = ['Lutrain']	| Number o

self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.129 sec/step (rd 2.0% | fw 37.0% | bw 61.0%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.142 sec/step (rd 2.2% | fw 37.1% | bw 60.7%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.128 sec/step (rd 2.1% | fw 36.9% | bw 61.1%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.135 sec/step (rd 2.1% | fw 37.1% | bw 60.9%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.140 sec/step (rd 2.0% | fw 37.2% | bw 60.8%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.135 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.135 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.127 sec/step (rd 2.1% | fw 36.6% | bw 61.3%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.137 sec/step (rd 2.2% | fw 36.8% | bw 61.0%))
self.tr_set: 881at | Loss - 0.00 | Grad. Norm - 0.00 | 0.129 sec/step (rd

self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.140 sec/step (rd 2.1% | fw 37.2% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.130 sec/step (rd 2.1% | fw 36.8% | bw 61.1%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.135 sec/step (rd 2.1% | fw 37.0% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.143 sec/step (rd 2.2% | fw 37.1% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.127 sec/step (rd 2.1% | fw 37.0% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.134 sec/step (rd 2.1% | fw 37.1% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.148 sec/step (rd 2.1% | fw 37.3% | bw 60.6%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.120 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.157 sec/step (rd 2.2% | fw 37.1% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.137 se

self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.133 sec/step (rd 2.2% | fw 36.8% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.137 sec/step (rd 2.1% | fw 36.8% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.141 sec/step (rd 2.1% | fw 37.1% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.129 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.142 sec/step (rd 2.1% | fw 37.2% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.147 sec/step (rd 2.1% | fw 37.2% | bw 60.6%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.135 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.140 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.137 sec/step (rd 2.2% | fw 37.1% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.138 se

self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.139 sec/step (rd 2.2% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.150 sec/step (rd 2.1% | fw 37.1% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.137 sec/step (rd 2.2% | fw 36.9% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.134 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.135 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.127 sec/step (rd 2.1% | fw 36.7% | bw 61.1%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.139 sec/step (rd 2.1% | fw 37.2% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.144 sec/step (rd 2.1% | fw 37.3% | bw 60.6%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.136 sec/step (rd 2.2% | fw 37.2% | bw 60.6%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.144 se

...

self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.142 sec/step (rd 2.3% | fw 37.0% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.131 sec/step (rd 2.2% | fw 36.8% | bw 61.1%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.153 sec/step (rd 2.2% | fw 36.9% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.123 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.144 sec/step (rd 2.1% | fw 37.0% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.137 sec/step (rd 2.2% | fw 36.8% | bw 61.1%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.130 sec/step (rd 2.2% | fw 36.7% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.134 sec/step (rd 2.1% | fw 36.9% | bw 61.1%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.133 sec/step (rd 2.2% | fw 36.8% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.139 se

self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.143 sec/step (rd 2.1% | fw 37.0% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.137 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.133 sec/step (rd 2.1% | fw 37.0% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.142 sec/step (rd 2.1% | fw 37.3% | bw 60.7%))
self.tr_set: 881d step - 2/2 0.00 | Grad. Norm - 0.00 | 0.130 sec/step (rd 2.0% | fw 37.0% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.146 sec/step (rd 2.2% | fw 37.1% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.138 sec/step (rd 2.3% | fw 37.0% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.143 sec/step (rd 2.1% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.126 sec/step (rd 2.3% | fw 36.7% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.131 se

self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.131 sec/step (rd 2.2% | fw 36.8% | bw 61.1%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.128 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.140 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.148 sec/step (rd 2.2% | fw 37.1% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.127 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.129 sec/step (rd 2.2% | fw 36.9% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.135 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.124 sec/step (rd 2.1% | fw 36.8% | bw 61.1%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.139 sec/step (rd 2.1% | fw 37.0% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.127 se

self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.124 sec/step (rd 2.2% | fw 37.2% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.141 sec/step (rd 2.2% | fw 36.9% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.131 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.142 sec/step (rd 2.1% | fw 37.2% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.140 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.147 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.138 sec/step (rd 2.1% | fw 37.2% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.142 sec/step (rd 2.1% | fw 37.2% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.139 sec/step (rd 2.1% | fw 37.1% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.138 se

self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.137 sec/step (rd 2.1% | fw 37.0% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.129 sec/step (rd 2.0% | fw 37.1% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.132 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.130 sec/step (rd 2.0% | fw 37.1% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.145 sec/step (rd 2.2% | fw 37.2% | bw 60.7%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.125 sec/step (rd 2.1% | fw 36.9% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.130 sec/step (rd 2.3% | fw 36.8% | bw 61.0%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.136 sec/step (rd 2.1% | fw 37.0% | bw 60.9%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.138 sec/step (rd 2.2% | fw 37.0% | bw 60.8%))
self.tr_set: 881tat | Loss - 0.00 | Grad. Norm - 0.00 | 0.147 se

In [14]:
print('asd')

asd
